In [1]:
import pandas as pd
#load
df = pd.read_csv("data/resale-flat-prices/merged_prices.csv")
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0


In [2]:
#drop duplicates
print(df.shape)
df.drop_duplicates(inplace=True)
df.shape

(883064, 10)


(881129, 10)

In [3]:
#check null value and dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 881129 entries, 0 to 883063
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                881129 non-null  object 
 1   town                 881129 non-null  object 
 2   flat_type            881129 non-null  object 
 3   block                881129 non-null  object 
 4   street_name          881129 non-null  object 
 5   storey_range         881129 non-null  object 
 6   floor_area_sqm       881129 non-null  float64
 7   flat_model           881129 non-null  object 
 8   lease_commence_date  881129 non-null  int64  
 9   resale_price         881129 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 73.9+ MB


In [4]:
df.flat_type.value_counts()

4 ROOM              332514
3 ROOM              284186
5 ROOM              185076
EXECUTIVE            66934
2 ROOM               10626
1 ROOM                1261
MULTI GENERATION       279
MULTI-GENERATION       253
Name: flat_type, dtype: int64

In [5]:
df['flat_model'] = df['flat_model'].str.upper()
df.flat_model.value_counts()

MODEL A                   247016
IMPROVED                  230865
NEW GENERATION            183416
SIMPLIFIED                 55828
PREMIUM APARTMENT          42115
STANDARD                   41379
APARTMENT                  34122
MAISONETTE                 28606
MODEL A2                    9641
DBSS                        2800
MODEL A-MAISONETTE          2003
ADJOINED FLAT               1183
TERRACE                      675
MULTI GENERATION             532
TYPE S1                      382
TYPE S2                      197
IMPROVED-MAISONETTE          119
PREMIUM APARTMENT LOFT        94
PREMIUM MAISONETTE            86
2-ROOM                        67
3GEN                           3
Name: flat_model, dtype: int64

In [6]:
#remove discontiued flat models or non-std flats for apple to apple comparison later
df_std = df.where(df.flat_model.isin(['MODEL A', 'IMPROVED', 'NEW GENERATION', 'SIMPLIFIED'
                                      , 'PREMIUM APARTMENT', 'STANDARD', 'APARTMENT', 'MODEL A2', '2-ROOM']))
df_std.dropna(inplace=True)
df_std.flat_type.value_counts()

4 ROOM       330755
3 ROOM       283226
5 ROOM       180695
EXECUTIVE     37887
2 ROOM        10625
1 ROOM         1261
Name: flat_type, dtype: int64

In [7]:
#treat masionnettes seaprately due to their distinct features and possibly different pricing from executive flat
df_mansionette = df.where(df.flat_model.isin(['MAISONETTE', 'MODEL A-MAISONETTE', 
                                      'IMPROVED-MAISONETTE', 'PREMIUM MAISONETTE']))
df_mansionette.dropna(inplace=True)
df_mansionette.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price
322,1990-01,BEDOK,5 ROOM,722,BEDOK RESERVOIR RD,07 TO 09,139.0,MODEL A-MAISONETTE,1984.0,170000.0
334,1990-01,BEDOK,EXECUTIVE,725,BEDOK RESERVOIR RD,01 TO 03,151.0,MAISONETTE,1984.0,195000.0
403,1990-01,BUKIT BATOK,EXECUTIVE,223,BT BATOK EAST AVE 3,04 TO 06,149.0,MAISONETTE,1985.0,160000.0
404,1990-01,BUKIT BATOK,EXECUTIVE,223,BT BATOK EAST AVE 3,01 TO 03,149.0,MAISONETTE,1985.0,160000.0
405,1990-01,BUKIT BATOK,EXECUTIVE,221,BT BATOK EAST AVE 3,10 TO 12,149.0,MAISONETTE,1985.0,160000.0


In [8]:
#relassify masionnettes flat type and put it back
df_mansionette['flat_type'] = 'MAISONETTE'
df = pd.concat([df_std, df_mansionette])

In [9]:
#check population size
df.flat_type.value_counts()

4 ROOM        330755
3 ROOM        283226
5 ROOM        180695
EXECUTIVE      37887
MAISONETTE     30814
2 ROOM         10625
1 ROOM          1261
Name: flat_type, dtype: int64

In [10]:
#drop flats due to insufficient samples to reduce bias or to be modelled separately
#executive and masionette maintained due to Section 1 Question B
#df_common = df.where(~df.flat_type.isin(["1 ROOM", "2 ROOM"]))
#df_common.dropna(inplace=True)
df_common = df.copy(deep=True)

In [11]:
#check sqm for each flat type
df_common.groupby('flat_type').floor_area_sqm.describe()

,count,mean,std,min,25%,50%,75%,max
flat_type,,,,,,,,
1 ROOM,1261.0,30.278351,0.999722,28.0,29.0,31.0,31.0,31.0
2 ROOM,10625.0,45.401336,3.801130,34.0,43.0,45.0,47.0,67.0
3 ROOM,283226.0,68.291674,6.232298,46.0,65.0,67.0,73.0,137.0
4 ROOM,330755.0,96.589313,8.305113,70.0,91.0,97.0,104.0,135.0
5 ROOM,180695.0,120.677512,6.879856,99.0,117.0,121.0,123.0,157.0
EXECUTIVE,37887.0,145.455750,11.613064,124.0,141.0,144.0,148.0,221.0
MAISONETTE,30814.0,147.878023,5.529556,133.0,146.0,147.0,150.0,243.0


In [12]:
df_common['address'] = df_common.block + " " + df_common.street_name
df_big3rm = df_common.where((df_common.flat_type=="3 ROOM") & (df_common.floor_area_sqm>100))
df_big3rm.dropna(inplace=True)
df_big3rm.floor_area_sqm.value_counts()

103.0    131
107.0     23
137.0     15
104.0     10
128.0      1
115.0      1
Name: floor_area_sqm, dtype: int64

In [13]:
df_big3rm137 = df_big3rm.where(df_big3rm.floor_area_sqm==137)
df_big3rm137.dropna(inplace=True)
df_big3rm137.address.value_counts()

4 SAGO LANE    15
Name: address, dtype: int64

In [14]:
#compare with the rest of "4 SAGO LANE"
df_compare = df_common.where((df_common.flat_type=="3 ROOM") & (df_common.address=="4 SAGO LANE"))
df_compare.dropna(inplace=True)
df_compare.floor_area_sqm.value_counts()
#likely adjoined flats instead

67.0     132
57.0      46
137.0     15
128.0      1
Name: floor_area_sqm, dtype: int64

In [15]:
df_big3rm103 = df_big3rm.where(df_big3rm.floor_area_sqm==103)
df_big3rm103.dropna(inplace=True)
df_big3rm103.address.value_counts()

14 KG ARANG RD           73
12 KG ARANG RD           56
210 NEW UPP CHANGI RD     2
Name: address, dtype: int64

In [16]:
#compare with the rest of "14 KG ARANG RD"
df_compare = df_common.where((df_common.flat_type=="3 ROOM") & (df_common.address=="14 KG ARANG RD"))
df_compare.dropna(inplace=True)
df_compare.floor_area_sqm.value_counts()
#likely true outlier

103.0    73
107.0    10
Name: floor_area_sqm, dtype: int64

In [17]:
#compare with the rest of "12 KG ARANG RD"
df_compare = df_common.where((df_common.flat_type=="3 ROOM") & (df_common.address=="12 KG ARANG RD"))
df_compare.dropna(inplace=True)
df_compare.floor_area_sqm.value_counts()
#likely true outlier

103.0    56
107.0    13
Name: floor_area_sqm, dtype: int64

In [18]:
#compare with the rest of "210 NEW UPP CHANGI RD"
df_compare = df_common.where((df.flat_type=="3 ROOM") & (df_common.address=="210 NEW UPP CHANGI RD"))
df_compare.dropna(inplace=True)
df_compare.floor_area_sqm.value_counts()
#likely adjoined flats

68.0     32
83.0      5
94.0      3
103.0     2
Name: floor_area_sqm, dtype: int64

In [19]:
#only left 104 and 115 sqm 3-rooms not checked. To drop since only 11 units.
#cleanup the 3 room
df_3rm = df_common.where((df_common.flat_type=="3 ROOM") 
                      & ((df_common.floor_area_sqm<=100)
                          | (df_common.floor_area_sqm==103)
                          | (df_common.floor_area_sqm==107)))
df_3rm.dropna(inplace=True)
df_3rm.floor_area_sqm.describe()
#non-appreciable improvement in statistics

count    283199.000000
mean         68.286398
std           6.207242
min          46.000000
25%          65.000000
50%          67.000000
75%          73.000000
max         107.000000
Name: floor_area_sqm, dtype: float64

In [20]:
df_common = df_common.where(~(df_common.flat_type=="3 ROOM") )
df_common.dropna(inplace=True)
df_common = pd.concat([df_common, df_3rm])

In [21]:
df_common.town.value_counts()

TAMPINES           76126
YISHUN             66257
BEDOK              64040
JURONG WEST        63493
WOODLANDS          61868
ANG MO KIO         49784
HOUGANG            48104
BUKIT BATOK        41929
CHOA CHU KANG      36247
BUKIT MERAH        32562
PASIR RIS          31955
TOA PAYOH          29701
SENGKANG           29094
QUEENSTOWN         27247
GEYLANG            26984
CLEMENTI           26720
BUKIT PANJANG      26298
KALLANG/WHAMPOA    24877
JURONG EAST        23889
SERANGOON          22051
BISHAN             20327
PUNGGOL            17129
SEMBAWANG          12143
MARINE PARADE       7736
CENTRAL AREA        6189
BUKIT TIMAH         2423
LIM CHU KANG          63
Name: town, dtype: int64

In [22]:
df_common.storey_range.value_counts()

04 TO 06    221052
07 TO 09    199392
01 TO 03    177283
10 TO 12    169325
13 TO 15     57234
16 TO 18     21843
19 TO 21     10354
22 TO 24      6698
25 TO 27      2906
01 TO 05      2688
06 TO 10      2465
11 TO 15      1257
28 TO 30      1218
31 TO 33       347
34 TO 36       332
37 TO 39       285
16 TO 20       264
40 TO 42       124
21 TO 25        92
26 TO 30        39
43 TO 45        23
36 TO 40         7
46 TO 48         6
31 TO 35         2
Name: storey_range, dtype: int64

In [23]:
df_common.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,address
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977.0,9000.0,309 ANG MO KIO AVE 1
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977.0,6000.0,309 ANG MO KIO AVE 1
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977.0,8000.0,309 ANG MO KIO AVE 1
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977.0,6000.0,309 ANG MO KIO AVE 1
104,1990-01,ANG MO KIO,4 ROOM,211,ANG MO KIO AVE 3,04 TO 06,81.0,NEW GENERATION,1977.0,80000.0,211 ANG MO KIO AVE 3


In [24]:
#break 'month' into year and month
df_common['year_sold'] = df_common['month'].str[0:4].astype(int)
df_common['mth_sold'] = df_common['month'].str[5:].astype(int)
#check unique values
print(df_common.year_sold.unique())
df_common.mth_sold.unique()

[1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003
 2004 2005 2006 2007 2008 2009 2010 2011 2012 2015 2016 2017 2018 2019
 2020 2021 2022 2013 2014]


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [25]:
#create age column
df_common['age_sold'] = df_common['year_sold'] - df_common['lease_commence_date']

In [26]:
#cleanup unnecessary columns
df_common.drop(['month', 'address', 'lease_commence_date'], axis=1, inplace=True)
df_common.head()
#not examining resale_price for outliers due to inflation and extraneous factors

,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,resale_price,year_sold,mth_sold,age_sold
0,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,9000.0,1990,1,13.0
1,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,6000.0,1990,1,13.0
2,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,8000.0,1990,1,13.0
3,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,6000.0,1990,1,13.0
104,ANG MO KIO,4 ROOM,211,ANG MO KIO AVE 3,04 TO 06,81.0,NEW GENERATION,80000.0,1990,1,13.0


In [27]:
df_common.corr()

,floor_area_sqm,resale_price,year_sold,mth_sold,age_sold
floor_area_sqm,1.000000,0.604089,0.071747,0.002949,-0.360678
resale_price,0.604089,1.000000,0.642375,0.015474,0.037792
year_sold,0.071747,0.642375,1.000000,-0.015675,0.450838
mth_sold,0.002949,0.015474,-0.015675,1.000000,-0.022721
age_sold,-0.360678,0.037792,0.450838,-0.022721,1.000000


In [28]:
"""
In March 2017, then-Minister for National Development, Lawrence Wong,
saw fit to respond to the phenomenon of aging, short-lease HDB flats 
being sold at high prices, as reported in Lianhe Zaobao.
2018 rally - LHL addressed issue of expiring leases of HDBS.
"""
df_new = df_common.where((df_common.year_sold<=2016) & (df_common.age_sold>30.0))
df_new.dropna(inplace=True)
df_new.corr()
#age becomes negative correlation

,floor_area_sqm,resale_price,year_sold,mth_sold,age_sold
floor_area_sqm,1.000000,0.766909,0.335743,0.000792,-0.252579
resale_price,0.766909,1.000000,0.655137,0.029434,0.059075
year_sold,0.335743,0.655137,1.000000,-0.000600,0.314233
mth_sold,0.000792,0.029434,-0.000600,1.000000,0.002393
age_sold,-0.252579,0.059075,0.314233,0.002393,1.000000


In [29]:
df_new = df_common.where((df_common.year_sold==2016) & (df_common.age_sold>30.0))
df_new.dropna(inplace=True)
df_new.corr()

,floor_area_sqm,resale_price,year_sold,mth_sold,age_sold
floor_area_sqm,1.000000,0.815663,NaN,-0.010381,-0.392902
resale_price,0.815663,1.000000,NaN,-0.010367,-0.165694
year_sold,NaN,NaN,NaN,NaN,NaN
mth_sold,-0.010381,-0.010367,NaN,1.000000,0.011318
age_sold,-0.392902,-0.165694,NaN,0.011318,1.000000


In [30]:
df_new = df_common.where((df_common.year_sold>2017) & (df_common.age_sold>30.0))
df_new.dropna(inplace=True)
df_new.corr()
#effect of ageing HDBs becomes more pronounced

,floor_area_sqm,resale_price,year_sold,mth_sold,age_sold
floor_area_sqm,1.000000,0.823611,0.062432,-0.004594,-0.441178
resale_price,0.823611,1.000000,0.225199,-0.001544,-0.259719
year_sold,0.062432,0.225199,1.000000,-0.142917,0.185778
mth_sold,-0.004594,-0.001544,-0.142917,1.000000,-0.022549
age_sold,-0.441178,-0.259719,0.185778,-0.022549,1.000000


In [31]:
#save file
df_common.to_csv("data/resale-flat-prices/cleaned_prices.csv", index=False)